In [1]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Settings
dataset_name='citeseer'
args.dataset=dataset_name
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)
nodesize = features.shape[0]

# Some preprocessing
features_tmp=features.copy()
features = preprocess_features(features).A
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
# Setup Surrogate model
idx_train=np.array(np.where(train_mask==1)).tolist()[0]
idx_val=np.array(np.where(val_mask==1)).tolist()[0]
idx_unlabeled=np.array(np.where(test_mask==1)).tolist()[0]
surrogate = GCN(nfeat=features.shape[1], nclass=single_label.max().item()+1,
                nhid=256, dropout=0, with_relu=False, with_bias=False, device='cpu').to('cpu')
surrogate.fit(features, adj, single_label, idx_train, idx_val, patience=100)
# Setup Attack Model
model = Metattack(surrogate, nnodes=adj.shape[0], feature_shape=features.shape,
        attack_structure=True, attack_features=False, device='cpu', lambda_=0).to('cpu')
# Attack
model.attack(features, adj, single_label, idx_train, idx_unlabeled, n_perturbations=10, ll_constraint=False)
modified_adj = model.modified_adj
# print(adj)
# print("shiy")
# print(modified_adj)

/home/zsp/ljx/PTDNet/utils.py:221: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()
Perturbing graph:   0%|          | 0/10 [00:00<?, ?it/s]

GCN loss on unlabled data: 1.694111704826355
GCN acc on unlabled data: 0.668
attack loss: 1.6872614622116089


Perturbing graph:  10%|█         | 1/10 [07:06<1:03:55, 426.17s/it]

GCN loss on unlabled data: 1.6943415403366089
GCN acc on unlabled data: 0.661
attack loss: 1.6871219873428345


Perturbing graph:  20%|██        | 2/10 [14:06<56:24, 423.02s/it]  

...

Perturbing graph:  80%|████████  | 8/10 [56:32<14:15, 427.59s/it]

GCN loss on unlabled data: 1.7052613496780396
GCN acc on unlabled data: 0.652
attack loss: 1.6984875202178955


Perturbing graph:  90%|█████████ | 9/10 [1:03:45<07:09, 429.06s/it]

GCN loss on unlabled data: 1.7043148279190063
GCN acc on unlabled data: 0.666
attack loss: 1.6973055601119995


Perturbing graph: 100%|██████████| 10/10 [1:10:46<00:00, 424.66s/it]


In [2]:
modified_adj=sp.csr_array(modified_adj.int())

In [3]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Settings
args.dataset=dataset_name
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)

nodesize = features.shape[0]

# Some preprocessing
features = preprocess_features(features)
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)



best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000
import time
begin = time.time()

model = PTDNetGCN(input_dim=features.shape[1], output_dim=y_train.shape[1])
model.set_fea_adj(np.array(range(adj.shape[0])), features_tensor, adj_tensor)

best_epoch = 0
curr_step = 0
best_val_acc = 0

for epoch in range(args.epochs):
    temperature = max(0.05,args.init_temperature * pow(args.temperature_decay, epoch))
    with tf.GradientTape() as tape:
        preds = []
        for l in range(args.outL):
            output = model.call(temperature,training=True)
            preds.append(tf.expand_dims(output,0))
        all_preds = tf.concat(preds,axis=0)
        mean_preds = tf.reduce_mean(preds,axis=0)
        consistency_loss = tf.nn.l2_loss(mean_preds-all_preds)

        cross_loss = masked_softmax_cross_entropy(mean_preds, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        lossl0 = model.lossl0(temperature)
        #nuclear = model.my_nuclear()
        nuclear = model.nuclear()
        loss = cross_loss + args.weight_decay*lossL2 + args.lambda1*lossl0 + args.lambda3*nuclear + args.coff_consis*consistency_loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call(None, training=False)
    edges_volumn = tf.reduce_sum(model.maskes[0])
    print('edge_vol',edges_volumn.numpy())

    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)
    if val_acc > best_val_acc:
        curr_step = 0
        best_epoch = epoch
        best_val_acc = val_acc
        best_val_loss= val_loss
        if val_acc>best_val_acc_trail:
            best_test_acc = test_acc
            best_val_acc_trail = val_acc
    else:
        curr_step +=1
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(train_acc), "val_acc=", "{:.5f}".format(val_acc),"best_val_acc_trail=", "{:.5f}".format(best_val_acc_trail),
      "test_acc=", "{:.5f}".format(best_test_acc))

    if curr_step > args.early_stop:
        print("Early stopping...")
        break
    end = time.time()
    print('time ',(end-begin))


/home/zsp/ljx/PTDNet/utils.py:221: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


edge_vol 4651.329
Epoch: 0001 train_loss= 1.79180 val_loss= 1.79106 train_acc= 0.45833 val_acc= 0.27600 best_val_acc_trail= 0.27600 test_acc= 0.26900
time  9.325296640396118
edge_vol 4639.838
Epoch: 0002 train_loss= 1.78934 val_loss= 1.79031 train_acc= 0.74167 val_acc= 0.33600 best_val_acc_trail= 0.33600 test_acc= 0.35000
time  11.080352544784546
edge_vol 4627.9053
Epoch: 0003 train_loss= 1.78680 val_loss= 1.78953 train_acc= 0.86667 val_acc= 0.39800 best_val_acc_trail= 0.39800 test_acc= 0.41000
time  12.685429096221924
edge_vol 4615.549
Epoch: 0004 train_loss= 1.78426 val_loss= 1.78870 train_acc= 0.90833 val_acc= 0.43800 best_val_acc_trail= 0.43800 test_acc= 0.46800
time  14.305488109588623
edge_vol 4602.8545
Epoch: 0005 train_loss= 1.78158 val_loss= 1.78781 train_acc= 0.91667 val_acc= 0.48400 best_val_acc_trail= 0.48400 test_acc= 0.51000
time  16.12254047393799
edge_vol 4589.9043
Epoch: 0006 train_loss= 1.77890 val_loss= 1.78684 train_acc= 0.92500 val_acc= 0.51600 best_val_acc_trail= 

In [4]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Settings
args.dataset=dataset_name

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)

nodesize = features.shape[0]

# Some preprocessing
features = preprocess_features(features)
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)



best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000
import time
begin = time.time()

model = PTDNetGCN(input_dim=features.shape[1], output_dim=y_train.shape[1])
model.set_fea_adj(np.array(range(adj.shape[0])), features_tensor, adj_tensor)

best_epoch = 0
curr_step = 0
best_val_acc = 0

for epoch in range(args.epochs):
    temperature = max(0.05,args.init_temperature * pow(args.temperature_decay, epoch))
    with tf.GradientTape() as tape:
        preds = []
        for l in range(args.outL):
            output = model.call(temperature,training=True)
            preds.append(tf.expand_dims(output,0))
        all_preds = tf.concat(preds,axis=0)
        mean_preds = tf.reduce_mean(preds,axis=0)
        consistency_loss = tf.nn.l2_loss(mean_preds-all_preds)

        cross_loss = masked_softmax_cross_entropy(mean_preds, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        lossl0 = model.lossl0(temperature)
        nuclear = model.my_nuclear()
        #nuclear = model.nuclear()
        loss = cross_loss + args.weight_decay*lossL2 + args.lambda1*lossl0 + args.lambda3*nuclear + args.coff_consis*consistency_loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call(None, training=False)
    edges_volumn = tf.reduce_sum(model.maskes[0])
    print('edge_vol',edges_volumn.numpy())

    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)
    if val_acc > best_val_acc:
        curr_step = 0
        best_epoch = epoch
        best_val_acc = val_acc
        best_val_loss= val_loss
        if val_acc>best_val_acc_trail:
            best_test_acc = test_acc
            best_val_acc_trail = val_acc
    else:
        curr_step +=1
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(train_acc), "val_acc=", "{:.5f}".format(val_acc),"best_val_acc_trail=", "{:.5f}".format(best_val_acc_trail),
      "test_acc=", "{:.5f}".format(best_test_acc))

    if curr_step > args.early_stop:
        print("Early stopping...")
        break
    end = time.time()
    print('time ',(end-begin))


edge_vol 4669.12
Epoch: 0001 train_loss= 1.79200 val_loss= 1.79143 train_acc= 0.38333 val_acc= 0.20000 best_val_acc_trail= 0.20000 test_acc= 0.15900
time  5.212229251861572
edge_vol 4667.5474
Epoch: 0002 train_loss= 1.78948 val_loss= 1.79066 train_acc= 0.70000 val_acc= 0.29600 best_val_acc_trail= 0.29600 test_acc= 0.24800
time  6.165233373641968
edge_vol 4663.0386
Epoch: 0003 train_loss= 1.78699 val_loss= 1.78988 train_acc= 0.88333 val_acc= 0.35600 best_val_acc_trail= 0.35600 test_acc= 0.32400
time  7.438089847564697
edge_vol 4657.0166
Epoch: 0004 train_loss= 1.78444 val_loss= 1.78906 train_acc= 0.93333 val_acc= 0.41000 best_val_acc_trail= 0.41000 test_acc= 0.39600
time  8.465310096740723
edge_vol 4650.0146
Epoch: 0005 train_loss= 1.78179 val_loss= 1.78818 train_acc= 0.95000 val_acc= 0.46800 best_val_acc_trail= 0.46800 test_acc= 0.44500
time  9.648217678070068
edge_vol 4642.327
Epoch: 0006 train_loss= 1.77923 val_loss= 1.78723 train_acc= 0.95000 val_acc= 0.51200 best_val_acc_trail= 0.5

/home/zsp/ljx/PTDNet/models.py:14: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


edge_vol 9248.0
Epoch: 0318 train_loss= 0.44401 val_loss= 1.14480 train_acc= 0.98333 val_acc= 0.71000 best_val_acc_trail= 0.71600 test_acc= 0.70800
time  340.0719335079193
edge_vol 9248.0
Epoch: 0319 train_loss= 0.44205 val_loss= 1.14327 train_acc= 0.98333 val_acc= 0.71000 best_val_acc_trail= 0.71600 test_acc= 0.70800
time  340.8832378387451
edge_vol 9248.0
Epoch: 0320 train_loss= 0.43985 val_loss= 1.14182 train_acc= 0.99167 val_acc= 0.71000 best_val_acc_trail= 0.71600 test_acc= 0.70800
time  341.91943097114563
edge_vol 9248.0
Epoch: 0321 train_loss= 0.43818 val_loss= 1.14028 train_acc= 0.99167 val_acc= 0.71000 best_val_acc_trail= 0.71600 test_acc= 0.70800
time  342.80277609825134
edge_vol 9248.0
Epoch: 0322 train_loss= 0.43706 val_loss= 1.13880 train_acc= 0.99167 val_acc= 0.71000 best_val_acc_trail= 0.71600 test_acc= 0.70800
time  344.1110498905182
edge_vol 9248.0
Epoch: 0323 train_loss= 0.43513 val_loss= 1.13729 train_acc= 0.99167 val_acc= 0.71000 best_val_acc_trail= 0.71600 test_acc

In [5]:
from config import args
import tensorflow as tf
import time
from utils import *
from models import GCN_dropedge
from metrics import *

# Settings
optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
args.dataset=dataset_name

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj

tuple_adj = sparse_to_tuple(adj.tocoo())
adj_tensor = tf.SparseTensor(*tuple_adj)

features = preprocess_features(features)

model = GCN_dropedge(input_dim=features.shape[1], output_dim=y_train.shape[1], adj=adj_tensor)


features_tensor = tf.convert_to_tensor(features,dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=tf.float32)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=tf.float32)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=tf.float32)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc = 0
best_val_loss = 10000


curr_step = 0
for epoch in range(args.epochs):

    with tf.GradientTape() as tape:
        output = model.call((features_tensor),training=True)
        cross_loss = masked_softmax_cross_entropy(output, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        loss = cross_loss + args.weight_decay*lossL2
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call((features_tensor), training=False)
    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)

    if val_acc > best_val_acc:
        curr_step = 0
        best_test_acc = test_acc
        best_val_acc = val_acc
        best_val_loss= val_loss
        # Print results

    else:
        curr_step +=1
    if curr_step > args.early_stop:
        print("Early stopping...")
        break

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(val_acc), "val_acc=", "{:.5f}".format(val_acc),
      "test_acc=", "{:.5f}".format(best_test_acc))

Epoch: 0001 train_loss= 1.79149 val_loss= 1.79123 train_acc= 0.24200 val_acc= 0.24200 test_acc= 0.27200
Epoch: 0002 train_loss= 1.78913 val_loss= 1.79051 train_acc= 0.33800 val_acc= 0.33800 test_acc= 0.37900
Epoch: 0003 train_loss= 1.78676 val_loss= 1.78976 train_acc= 0.44800 val_acc= 0.44800 test_acc= 0.45900
Epoch: 0004 train_loss= 1.78438 val_loss= 1.78897 train_acc= 0.50600 val_acc= 0.50600 test_acc= 0.50600
Epoch: 0005 train_loss= 1.78194 val_loss= 1.78811 train_acc= 0.54400 val_acc= 0.54400 test_acc= 0.53400
Epoch: 0006 train_loss= 1.77940 val_loss= 1.78719 train_acc= 0.57400 val_acc= 0.57400 test_acc= 0.55600
Epoch: 0007 train_loss= 1.77672 val_loss= 1.78618 train_acc= 0.59400 val_acc= 0.59400 test_acc= 0.57800
Epoch: 0008 train_loss= 1.77390 val_loss= 1.78508 train_acc= 0.61000 val_acc= 0.61000 test_acc= 0.60000
Epoch: 0009 train_loss= 1.77090 val_loss= 1.78390 train_acc= 0.61600 val_acc= 0.61600 test_acc= 0.60600
Epoch: 0010 train_loss= 1.76772 val_loss= 1.78264 train_acc= 0.6